In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [5]:
# read the csv
df = pd.read_csv('distribution.csv')
df.head()

label_matrix = np.array(df.values[:, 1:-1])

In [6]:
# label_matrix = label_matrix[:20, :]

In [7]:
print(label_matrix.shape)

(41, 4)


In [15]:
split_rate = 0.75
tolerance = 0.05
total_sum_label = np.array(df.iloc[:, 1:-1].sum())
total_sum_label = np.sum(label_matrix.astype(np.int64), axis=0)

In [16]:
total_sum_label

array([1106,  350,  389, 1990], dtype=int64)

In [23]:
def check_upper_unsatisfied(current_situation):
    upper = split_rate + tolerance
    current_situation = np.divide(current_situation, total_sum_label)
    return np.all(current_situation <= upper)

def check_lower_satisfied(current_situation):
    lower = split_rate - tolerance
    current_situation = np.divide(current_situation, total_sum_label)
    return np.all(current_situation >= lower)

def check_lower_unsatisfied(current_situation):
    lower = split_rate - tolerance
    current_situation = np.divide(current_situation, total_sum_label)
    return np.any(current_situation < lower)

In [35]:
def diff(current_situation, noise = False):
    # print(total_sum_label - current_situation)
    diff_num = total_sum_label * split_rate - current_situation
    if noise == True:
        diff_num = makeNoise(diff_num, scale = np.abs(np.mean(diff_num)/10))
    # if the diff is negative, increase its weight
    diff_num[diff_num > 0] *= 5
    return np.sum(diff_num)

def makeNoise(score, scale):
    gumbel_noise = np.random.gumbel(0, scale, size=score.shape) + score
    return gumbel_noise

In [25]:
makeNoise(np.array([50,50,100,100]), 20)

array([ 55.55135354,  58.5077074 ,  92.69979768, 124.59671125])

In [26]:
import threading
import numpy as np
import time

class myThread(threading.Thread):
  def __init__(self, group, target, name, args, kwargs, daemon) -> None:
    super().__init__(group, target, name, args, kwargs, daemon=daemon)
    self.result = None
    self.target = target
    self.args = args
    self.kwargs = kwargs

  def run(self):
    self.result = self.target(*self.args, **self.kwargs)
    print(f"{self.name} finished at {time.ctime()}")

def add_arrays(arr1, arr2):
  return np.add(arr1, arr2)

threads = []

In [27]:
index = np.arange(label_matrix.shape[0])
print(index)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40]


In [28]:
# arrays = [np.random.randint(1, 10, size=3) for _ in range(200)]
# for i in range(100):
#   # 从arrays中取出两个数组作为参数
#   arr1 = arrays[2 * i]
#   arr2 = arrays[2 * i + 1]
#   # 创建一个线程的实例，传入函数和数组作为参数
#   thread = myThread(None, add_arrays, f"thread_{i}", (arr1, arr2), {}, None)
#   # 将线程的实例添加到列表中
#   threads.append(thread)

# # 循环列表中的每个线程
# for thread in threads:
#   # 启动线程
#   thread.start()

# # 循环列表中的每个线程
# for thread in threads:
#   # 等待线程结束
#   thread.join()
#   # 打印线程的名字和结果
#   print(f"{thread.name}: {thread.result}")


In [29]:
from heapq import heappush, heappop, heapify
def greedy_method(label_matrix):
    end = False
    current_result = np.zeros(label_matrix.shape[1]).astype(np.int64)
    index = list(range(label_matrix.shape[0]))
    result_idx = []
    label_matrix = label_matrix.astype(np.int64)
    while not end:
        # new a queue
        heap = []
        for i in index:
            diff_result = diff(current_result + label_matrix[i])
            heappush(heap, (diff_result, i))
        
        # get the best result
        # print(heap)
        # if all are negative, then end
        if heap[0][0] < 0:
            end = True
            break
        best_result = heappop(heap)
        current_result += label_matrix[best_result[1]]
        index.remove(best_result[1])
        print(f"best_result: {best_result}")
        print(f"label matrix: {label_matrix[best_result[1]]}")
        print(f"current_result: {current_result}")
        result_idx.append(best_result[1])
        print(result_idx)
    return current_result, result_idx

In [40]:

current_result, result_idx = greedy_method(label_matrix)

best_result: (10456.451521207633, 36)
label matrix: [  0   0   0 725]
current_result: [  0   0   0 725]
[36]
best_result: (9260.804386895634, 40)
label matrix: [130   0   0 229]
current_result: [130   0   0 954]
[36, 40]
best_result: (7517.937763044056, 34)
label matrix: [ 90   0   0 117]
current_result: [ 220    0    0 1071]
[36, 40, 34]
best_result: (6507.365377223946, 35)
label matrix: [ 47   0 199  30]
current_result: [ 267    0  199 1101]
[36, 40, 34, 35]
best_result: (5592.103229022665, 37)
label matrix: [55 17 66  0]
current_result: [ 322   17  265 1101]
[36, 40, 34, 35, 37]
best_result: (4984.4612353126595, 29)
label matrix: [143   0   0   0]
current_result: [ 465   17  265 1101]
[36, 40, 34, 35, 37, 29]
best_result: (4518.140889876255, 13)
label matrix: [81 52  0  0]
current_result: [ 546   69  265 1101]
[36, 40, 34, 35, 37, 29, 13]
best_result: (3665.1432240032136, 33)
label matrix: [  0   0   0 147]
current_result: [ 546   69  265 1248]
[36, 40, 34, 35, 37, 29, 13, 33]
best_

In [43]:
print(np.divide(current_result, total_sum_label))
print(sorted(result_idx))

[0.74231465 0.77428571 0.81491003 0.75226131]
[2, 8, 9, 13, 17, 21, 22, 25, 28, 29, 32, 33, 34, 35, 36, 37, 40]


In [1]:
def diff2(current_situation):
    # print(total_sum_label - current_situation)
    noise = False
    diff_num = total_sum_label * split_rate - current_situation
    if noise == True:
        diff_num = makeNoise(diff_num, scale = np.abs(np.mean(diff_num)/10))
    # if the diff is negative, increase its weight
    
    # normalize the diff
    
    diff_num = np.divide(diff_num, total_sum_label)
    
    diff_num[diff_num > 0] = np.exp(diff_num[diff_num > 0])
    diff_num[diff_num < 0] = -np.exp(diff_num[diff_num < 0])
    return np.sum(diff_num)

In [147]:
# method 2

from itertools import combinations as comb

def greedy_method2(label_matrix):
    end = False
    
    combination_list = comb(range(label_matrix.shape[0]), 2)
    # print(list(combination_list))
    # print(label_matrix)
    k = 1000
    counter = 0
    result = None
    while not end:
        counter += 1
        queue = []
        
        for i in list(combination_list):
            # print(i)
            # print(label_matrix[i, :])
            # print(np.sum(label_matrix[i, :], axis=0))
            diff_result = diff2(np.sum(label_matrix[i, :], axis=0).astype(np.int64))
            heappush(queue, (diff_result, i))
            # print(queue)
        print(len(queue))
        # print(queue)
        print(f"best result: {queue[0]}")
        if queue[0][0] < 0 or len(queue) < 50:
            end = True
            result = queue
            break
        k_top_index = [heappop(queue)[1] for _ in range(np.min((k, len(queue))))]
        # print(k_top_index)
        
        combination_list = list(comb(k_top_index, 2))
        # print(combination_list)
        # remove the duplicated index tuple, i.e. (1,2), (2,3) -> (1,2,3)
        combination_list = list(set([tuple(sorted(set(a + b))) for a, b in combination_list]))
        print(f"{counter}th iteration, len of combination_list: {len(combination_list)}")
        
        
        if counter == 10: break
        
        # print(combination_list)
        
    return result
        
        # get the best result

In [150]:
result = greedy_method2(label_matrix)
print(len(result))

820
best result: (6.86378694107514, (35, 36))
1th iteration, len of combination_list: 111930
111930
best result: (4.139780910671986, (2, 35, 36, 37))
2th iteration, len of combination_list: 174041
174041
best result: (3.005722101272507, (2, 8, 13, 34, 35, 36, 37, 40))
3th iteration, len of combination_list: 66928
66928
best result: (0.5614655933254309, (2, 8, 9, 13, 21, 27, 28, 34, 35, 36, 37, 40))
4th iteration, len of combination_list: 51320
51320
best result: (-1.90318487753063, (2, 8, 9, 13, 17, 21, 22, 25, 28, 29, 32, 33, 34, 35, 36, 37, 40))
51320


In [152]:
# get the top 10 results from the heap
top_10 = [heappop(result) for _ in range(10)]

for i in top_10:
    # print(top_10)
    index = i[1]
    print(index)
    print(np.sum(label_matrix[index,:], axis=0))
    print(np.divide(np.sum(label_matrix[index,:], axis=0), total_sum_label))

(2, 8, 9, 13, 17, 21, 22, 25, 26, 28, 29, 32, 34, 35, 36, 37, 40)
[843.0 275.0 320.0 1385.0]
[0.7622061482820977 0.7857142857142857 0.8226221079691517
 0.6959798994974874]
(2, 8, 9, 11, 13, 17, 21, 22, 25, 28, 29, 32, 34, 35, 36, 37, 40)
[857.0 271.0 317.0 1369.0]
[0.7748643761301989 0.7742857142857142 0.8149100257069408
 0.6879396984924623]
(2, 3, 8, 9, 13, 17, 21, 23, 25, 28, 29, 32, 34, 35, 36, 37, 40)
[836.0 279.0 319.0 1382.0]
[0.755877034358047 0.7971428571428572 0.8200514138817481
 0.6944723618090453]
(2, 8, 9, 10, 13, 17, 21, 23, 25, 28, 29, 33, 34, 35, 36, 37, 40)
[767.0 271.0 317.0 1565.0]
[0.6934900542495479 0.7742857142857142 0.8149100257069408
 0.7864321608040201]
(1, 2, 8, 9, 13, 21, 23, 28, 29, 32, 33, 34, 35, 36, 37, 40)
[696.0 263.0 313.0 1493.0]
[0.6292947558770343 0.7514285714285714 0.8046272493573264
 0.750251256281407]
(1, 2, 8, 9, 10, 13, 21, 28, 29, 32, 33, 34, 35, 36, 37, 40)
[688.0 263.0 313.0 1501.0]
[0.6220614828209765 0.7514285714285714 0.8046272493573264
 0

In [3]:
import numpy as np
import pandas as pd
from heapq import heappush, heappop, heapify

class RandomGroupSplitter:
    def __init__(self, noise, force_bound, tolerance, split_rate):
        self.noise = noise
        self.force_bound = force_bound
        self.tolerance = tolerance
        self.split_rate = split_rate
        
    def get_one_combination(self, label_matrix):
        end = False
        current_result = np.zeros(label_matrix.shape[1]).astype(np.int64)
        index = list(range(label_matrix.shape[0]))
        result_idx = []
        label_matrix = label_matrix.astype(np.int64)
        while not end:
            # new a queue
            heap = []
            for i in index:
                diff_result = diff(current_result + label_matrix[i], noise= self.noise)
                heappush(heap, (diff_result, i))

            if heap[0][0] < 0:
                end = True
                break
            best_result = heappop(heap)
            current_result += label_matrix[best_result[1]]
            index.remove(best_result[1])
            print(f"best_result: {best_result}")
            print(f"label matrix: {label_matrix[best_result[1]]}")
            print(f"current_result: {current_result}")
            result_idx.append(best_result[1])
            print(result_idx)
        return current_result, result_idx
    
    def get_multi_combination():
        pass